<a href="https://colab.research.google.com/github/ch00226855/CMP414765Fall2022/blob/main/Week04_MultilinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 6
# Multilinear Regression

Last time we looked at a simple linear regression model $sales = \beta_0 + \beta_1\cdot\textit{TV advertising budget}$. More generally, a linear model makes a prediction by computing a weighted sum of their input features (plus a constant).

**Reading: Chapter 4**

## Multilinear Regression: Model Assumptions
**Model**:

$\hat{y} = \theta_0 + \theta_1x_1 + \theta_2x_2 +\cdots + \theta_nx_n$
1. $\hat{y}$ is the predicted value.
2. $n$ is the number of features.
3. $x_i$ is the i-th feature value.
4. $\theta_j$ is the j-th model parameter (associated with $x_j$).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Toy example
columns = ['Homework', 'Midterm', 'Final']
data = pd.DataFrame({
    "Homework": [95, 70, 80, 100, 70],
    "Midterm": [90, 60, 80, 80, 85],
    "Final": [93, 66, 85, 60, 90]
}, index=["Alice", "Bob", "Clare", "David", "Eve"])

data.head()

In this case:
- $x_1$ is the homework feature
- $x_2$ is the midterm feature
- $y$ is the final feature
- model is: $final = \theta_0 + \theta_1 * homework + \theta_2 * midterm$
- We need to come up with values for $\theta_0, \theta_1, \theta_2$ to complete the model.

**Objective**: Suppose that another student Fred has Homework score 85 and Midterm score 80. What is prediction of his final exam score?

## Multilinear Regression: Vectorized form

The multilinear model can also be written as:

**$\hat{y} = \theta\cdot\textbf{x}$**.
1. $\theta = (\theta_0, \theta_1, ..., \theta_n)$ is the paramter vector.
2. $\textbf{x} = (1, x_1, ..., x_n)$ is the feature vector.
3. The symbol $\cdot$ represents the inner-product of two vectors. For example, $(1, 2, 3)\cdot (4, 5, 6) = 1\times 4 + 2\times 5 + 3\times 6 = 32$.

**Why is the expression $\theta\cdot\textbf{x}$ equivalent to $\theta_0 + \theta_1x_1 + \theta_2x_2 +\cdots + \theta_nx_n$?**

In [ ]:
data[["Homework", "Midterm"]]

In [ ]:
# Let's apply the linear regression tool in sci-kit learn on the toy example
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(data[["Homework", "Midterm"]], data['Final']) # You can also use data[['Final']]


In [ ]:
# What is the prediction of Fred's final score?

# Create a dataframe with Fred's records
fred = pd.DataFrame([[85, 80]], columns=["Homework", "Midterm"], index=['Fred'])
# fred
model.predict(fred)

In [ ]:
# Model's prediction is 78. This looks safe. But how much variation the prediction has?
# We can use model's error on the training data as an indicator

data['Prediction'] = model.predict(data[['Homework', 'Midterm']])
data

In [ ]:
# Retrieve the estimated parameter values.
print("theta1, theta2:", model.coef_)
print("theta0:", model.intercept_)

Now, let's create a linear model that describes the relationship between `sales` and all three ways of advertising.

In [ ]:
url = "https://www.statlearning.com/s/Advertising.csv"
advertising = pd.read_csv(url, index_col=0)
advertising.head()

Proposed model: $sales = \theta_0 + \theta_1\cdot TV + \theta_2\cdot radio + \theta_3\cdot newspaper$.

In [ ]:
from sklearn.model_selection import train_test_split
training_set, test_set = train_test_split(advertising, test_size=0.1)
print(training_set.shape)
print(test_set.shape)

In [ ]:
model2 = LinearRegression()
model2.fit(training_set[['TV', 'radio', 'newspaper']], training_set['sales'])

In [ ]:
input_cols = ['TV', 'radio', 'newspaper']

In [ ]:
# The MSE of model2 on the training set
from sklearn.metrics import mean_squared_error
y_pred = model2.predict(training_set[input_cols])
training_MSE = mean_squared_error(training_set['sales'], y_pred)
print("MSE on the training set:", training_MSE)
# This MSE is better than the MSE achieved by a simple linear regression model.

In [ ]:
# The MSE of model2 on the test set
y_pred = model2.predict(test_set[input_cols])
test_MSE = mean_squared_error(test_set['sales'], y_pred)
print("MSE on the training set:", test_MSE)

## Multilinear Regression: Cost Function
In order to calculate the best value for each parameter, we need a **cost function** that evaluates the errors made by a give set of parameter values. Here we use the **mean squared error (MSE)** function as the cost function:

$J(\textbf{X}, \theta) = \frac{1}{m}\sum_{i=1}^{m}\big(\theta\cdot\textbf{x}^{(i)} - y^{(i)}\big)^2$

Here $(\textbf{x}^{(i)}, y^{(i)})$ represents the i-th training example

In [ ]:
# Calculate the MSE cost of the toy example for the parameter values given by sci-kit learn.



## Multilinear Regression: Training Algorithm 1
The value of $\theta$ that minimizes the cost function is given by the following **normal equation**:

$\hat{\theta} = \big(\textbf{X}^T\cdot\textbf{X}\big)^{-1}\cdot\textbf{X}^T\cdot\textbf{y}$.

1. $\textbf{X}$ is an $m\times (n+1)$ matrix whose i-th row is $\textbf{x}^{(i)}$.
$$\textbf{X} = \begin{pmatrix}
1 & x^{(1)}_1 & x^{(1)}_2 & \cdots & x^{(1)}_n \\
1 & x^{(2)}_1 & x^{(2)}_2 & \cdots & x^{(2)}_n \\
\vdots & \vdots &\vdots & \ddots & \vdots \\
1 & x^{(m)}_1 & x^{(m)}_2 & \cdots & x^{(m)}_n \\
\end{pmatrix}$$
2. $$\textbf{y} = \begin{pmatrix}y^{(1)} \\ \vdots \\ y^{(m)}\end{pmatrix}$$.
3. The cost function $J(\theta)$ also has a matrix expression
$$J(\theta) = \frac{1}{m}(\textbf{X}\cdot\theta - \textbf{y})^T\cdot (\textbf{X}\cdot\theta - \textbf{y})$$

In [ ]:
# Construct matrix X using np.hstack(), np.ones()
N = len(training_set) # N = training_set.shape[0]
col1 = np.ones([N, 1])
# print(col1)
col234 = training_set[['TV', 'radio', 'newspaper']].values
# print(col234)
X = np.hstack([col1, col234])
print(X[0:5, :])

In [ ]:
# Construct vector y
y = training_set[['sales']].values
print(y[:5])

In [ ]:
# Apply the normal equation to find theta
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})

np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

In [ ]:
# The results should be the same as the sklearn model
print(model2.coef_)
print(model2.intercept_)

## Multilinear Regression: Training Algorithm 2
The normal equation is not applicable when $\textbf{X}^T\cdot\textbf{X}$ is not invertible. It happens if:
- Several features are linearly dependent (for example, feature3 = feature1 + feature2)
- The number of features is greater than the number of training data (for example, DNA data)

When the matrix $\textbf{X}$ is too large, the normal equation may take too long to finish since it requires a matrix multiplication.

In these cases, we can use the **gradient descent** method to minimize the cost function instead.

Gradient descent with one variable ideally looks like this:
<img src="https://cdn-images-1.medium.com/max/1600/0*fU8XFt-NCMZGAWND." width="600">

Gradient descent with two variables ideally looks like this:
<img src="https://blog.paperspace.com/content/images/2019/09/F1-02.large.jpg" width="600">

Gradient descent is an iterative algorithm for finding the **local minimum** of a differentiable function.
- Choose an initial value of $\hat{\theta}$ and a **learning rate** $r$.
- For each iteration $k$, do:
$$\hat{\theta} \leftarrow \hat{\theta} - r\cdot\frac{\partial J(\hat{\theta})}{\partial \theta}.$$
- The partial derivative of the cost function is given by
$$
\frac{\partial J(\hat{\theta})}{\partial \theta} = \frac{2}{m}\cdot\textbf{X}^T\cdot(\textbf{X}\cdot\theta - \textbf{y}).
$$
- **Verify the formula of partial derivative asuuming there is one input feature.**

- End iteration if certain stop criteria is reached, such as:
    - Value of $\hat{\theta}$ becomes stable.
    - Certain iteration amount is reached.

In [ ]:
# Choose a random initial value for each parameter.
np.random.seed(1234) 
theta = np.random.randn(4, 1)
print(theta)

In [ ]:
def MSE_cost(theta, X, y):
    m = X.shape[0]
    M = X.dot(theta) - y
    return 1/m * M.T.dot(M)

print("Initial cost:", MSE_cost(theta, X, y))

In [ ]:
# Perform gradient descent once.
# Choose a learning rate r
r = 1e-5 # r = 0.00001

# 1. Calculate the gradient
m = len(training_set)
gradient = 2/m * X.T.dot(X.dot(theta) - y)
print(gradient)

# 2. Update the parameters
theta = theta - r * gradient
print(theta)


# 3. (optional) Show the MSE cost with new parameter values


MSE_cost(theta, X, y)

In [ ]:
# Perform gradient descent multiple times
r = 0.00001
np.random.seed(1234) 
theta = np.random.randn(4, 1)
print(theta)

MSEs = [] # store the intermediate MSEs
num_iterations = 50000

for i in range(num_iterations):

    gradient = 2/m * X.T.dot(X.dot(theta) - y)
    theta = theta - r * gradient

    if i % 1000 == 0:
        print("Iteration:", i)
        print("Gradient:\n", gradient)
        print("New theta:\n", theta)
        MSE = MSE_cost(theta, X, y)
        print("New cost:", MSE)
        MSEs.append(MSE[0][0])


In [ ]:
# Calculate the MSE with different parameter values

# Optimal parameter values:
from sklearn.metrics import mean_squared_error
y_pred = model2.predict(training_set[["TV", "radio", "newspaper"]])
MSE = mean_squared_error(training_set['sales'], y_pred)
print("Optimal MSE:", MSE)

In [ ]:
MSEs

In [ ]:
# Plot the learning curve.

plt.plot(range(len(MSEs)-2), MSEs[2:], 'b.--')

**Discussion**
1. Change $r$ to 0.000001 and 1. Observe the MSE curve.
2. Do the initial parameter values matter?
3. How to determine when to stop the iteration?